# 03: TypeKind — Declaration-Level Architectural Enforcement

Kind classifies **places** (directories and files). TypeKind classifies **types** (individual exported declarations).

The type annotation IS the instance — no `satisfies`, no `register()`, no companion const. When you write `export const f: Decider = ...`, the `: Decider` annotation is both the TypeScript type and the KindScript architectural declaration.

This notebook walks through:

**Part 1 — The Problem:** Directory-level constraints can't group exports within a directory

**Part 2 — TypeKind:** Define TypeKinds that wrap function types

**Part 3 — Composability:** Use TypeKind members inside a Kind with constraints

**Part 4 — Catch a Violation:** Introduce and detect a forbidden dependency

**Prerequisites:** Run `npx tsc` from the project root to compile the CLI.

In [ ]:
import { ksc, tree, copyFixture, writeFile, readFile, cleanup } from './lib.ts';
console.log("Setup complete.");

---

## Part 1: The Problem

Existing Kind constraints work at the directory level. `noDependency: [["domain", "infra"]]` means "no file in `domain/` can import from `infra/`."

But what if you want to constrain groups of **functions within the same directory**? For example, in event-sourced systems you might have:

- **Deciders** — pure functions that take a command and return events
- **Effectors** — side-effectful functions that react to events

Both live in the same module directory. You want to enforce: "no Decider may depend on an Effector." Directories can't express this — you need declaration-level grouping.

In [ ]:
const DEMO = copyFixture("typekind");
console.log("Demo project:", DEMO);

console.log("\nProject structure:");
await tree(DEMO);

---

## Part 2: Defining TypeKinds

TypeKind wraps a TypeScript type to give it architectural meaning:

```typescript
type DeciderFn = (command: Command) => Event[];
type Decider = TypeKind<"Decider", DeciderFn>;
```

`Decider` is now both a TypeScript type AND an architectural declaration. Any export annotated with `: Decider` is a TypeKind instance.

In [ ]:
console.log("=== src/types.ts ===");
console.log(readFile(DEMO, "src/types.ts"));

console.log("\n=== src/context.ts ===");
console.log(readFile(DEMO, "src/context.ts"));

The `context.ts` file defines:

1. **Two TypeKinds** — `Decider` wraps `DeciderFn`, `Effector` wraps `EffectorFn`
2. **A filesystem Kind** — `OrderModule` with two TypeKind members: `deciders` and `effectors`
3. **A constraint** — `noDependency: [["deciders", "effectors"]]` forbids Decider files from importing Effector files
4. **An instance** — `Instance<OrderModule, '.'>` maps the module to the current directory

The constraint means: "within this directory, collect all Decider-typed exports into one group and all Effector-typed exports into another. No Decider may reference an Effector."

---

## Part 3: TypeKind Instances

The type annotation IS the instance. Let's look at the actual function files:

In [ ]:
console.log("=== src/validate-order.ts (Decider) ===");
console.log(readFile(DEMO, "src/validate-order.ts"));

console.log("\n=== src/apply-discount.ts (Decider) ===");
console.log(readFile(DEMO, "src/apply-discount.ts"));

console.log("\n=== src/notify-order.ts (Effector) ===");
console.log(readFile(DEMO, "src/notify-order.ts"));

Notice:
- `validateOrder` and `applyDiscount` have type annotation `: Decider` — they're Decider instances
- `notifyOrder` has type annotation `: Effector` — it's an Effector instance
- Discovery is **syntactic** — the scanner checks the type annotation name, not structural compatibility
- A function typed as `: DeciderFn` (not TypeKind) would NOT be an instance, even though it's structurally identical

In [ ]:
console.log("=== Check: clean project ===");
await ksc("check", DEMO);

---

## Part 4: Catching a Violation

All good — no Decider imports from an Effector. Now let's break the rule: have `apply-discount.ts` (a Decider) import from `notify-order.ts` (an Effector).

In [ ]:
const cleanApplyDiscount = readFile(DEMO, "src/apply-discount.ts");

writeFile(DEMO, "src/apply-discount.ts", `import { notifyOrder } from './notify-order';
import type { DeciderFn } from './types';
import type { TypeKind } from 'kindscript';

type Decider = TypeKind<"Decider", DeciderFn>;

export const applyDiscount: Decider = (command) => {
  notifyOrder({ type: 'DiscountApplied', data: command });
  return [{ type: 'DiscountApplied', data: command }];
};
`);

console.log("Injected violation: apply-discount.ts now imports from notify-order.ts");
console.log("(A Decider importing from an Effector — forbidden by noDependency)\n");

await ksc("check", DEMO);

KindScript caught it:
- **KS70001: Forbidden dependency** — a file containing a Decider (`apply-discount.ts`) imported from a file containing an Effector (`notify-order.ts`)
- The `noDependency: [["deciders", "effectors"]]` constraint on `OrderModule` caught this
- The parser resolved TypeKind members: all Decider-typed exports in one group, all Effector-typed exports in another

**Key insight:** The binder and checker needed zero changes to support TypeKind. The `resolvedFiles` abstraction hides whether a member is a directory or a typed-export group. Existing plugins work unchanged.

In [ ]:
writeFile(DEMO, "src/apply-discount.ts", cleanApplyDiscount);
console.log("Restored clean version\n");
await ksc("check", DEMO);

In [ ]:
cleanup(DEMO);
console.log("Demo project cleaned up.");

---

## Summary

| Concept | How it works |
|---------|-------------|
| `TypeKind<N, T>` | Wraps a type with architectural meaning |
| Instance declaration | Type annotation IS the instance (`: Decider`) |
| Discovery | Syntactic — by explicit type name, not structural matching |
| Composability | TypeKind members inside Kind, with full constraint support |
| Resolution | Parser collects typed exports within parent scope |

**Key insight:** TypeKind and Kind compose naturally. The binder and checker needed zero changes — the `resolvedFiles` abstraction hides whether a member is a directory or a typed-export group.

**Reference:** See [Kind System documentation](../docs/02-kind-system.md) for the full TypeKind reference, and [docs/06-tutorial.md](../docs/06-tutorial.md) for a complete static walkthrough.